In [329]:
#loading the required packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import plotnine as p9 
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from scipy.stats.mstats import winsorize


In [330]:
#Fetching the data
raw_data = pd.read_csv('./data/tesco-dataset/train.csv')
print('The shape of the dataset :' + str(raw_data.shape))
raw_data.head()

The shape of the dataset :(320, 16)


,location_id,crime_rate,proportion_flats,proportion_nonretail,new_store,commercial_property,household_size,proportion_newbuilds,public_transport_dist,transport_availability,property_value,school_proximity,competitor_density,household_affluency,normalised_sales,county
0,464,17.600541,0.0,18.10,no,NaN,2.926,29.0,2.9084,All transport options,666,20.2,368.74,4.5325,-0.399933,c_40
1,504,0.603556,20.0,3.97,no,14.85,4.520,10.6,2.1398,Average transport options,264,13.0,388.37,1.8150,2.216308,c_80
2,295,0.606810,0.0,6.20,no,7.70,2.981,31.9,3.6715,Many transport options,307,17.4,378.35,2.9125,0.166920,c_53
3,187,0.012385,55.0,2.25,no,1.95,3.453,68.1,7.3073,No transport options,300,15.3,394.72,2.0575,-0.083804,c_65
4,193,0.016182,100.0,1.32,no,3.05,3.816,59.5,8.3248,Average transport options,256,15.1,392.90,0.9875,0.962693,c_97


# Plan for modelling (MVP):
* Usuals - > Train_test split, target variable split, 
1. Missing values for school_proximity and commercial_property - Start with median imputation and check the model accuracy
2. Categorical encoding
    1. Binary encoding for new_store feature
    2. Ordinal encoding for transport_availability feature
    3. Group counties to high, medium, low sales
3. Outlier handling if requred - not for MVP
4. Feature selection
5. Model development 

In [331]:
# train_test split
X=raw_data.drop("normalised_sales", axis=1)
y=raw_data[["normalised_sales"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [332]:
# Ordinal transformation
order=[['No transport options','Few transport options','Average transport options','Many transport options','All transport options'        
        ]]

In [333]:

# custom transformer

class CountySalesEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, high_threshold=0.66, low_threshold=0.33):
        # Thresholds to define 'high', 'medium', 'low' sales categories
        self.high_threshold = high_threshold
        self.low_threshold = low_threshold

    def fit(self, X, y=None):
        # Assuming 'X' is a DataFrame with a 'county' column and 'y' contains sales data
        # Calculate and store the average sales for each county
        if y is None:
            raise ValueError("y cannot be None. Please provide the sales data.")
        
        # Ensure X and y have the same length
        if len(X) != len(y):
            raise ValueError("The length of X and y must be the same.")
       
        # Calculate the average sales per county
        self.county_sales_averages = y.squeeze().groupby(X['county']).mean()
        return self

    def transform(self, X, y=None):
        # Check if fit has been called
        if not hasattr(self, "county_sales_averages"):
            raise AttributeError("fit has not been called. Please call fit before transform.")
        
        # Map each county to its average sales
        X['average_sales'] = X['county'].map(self.county_sales_averages)

        # Categorize based on the thresholds
        categories = pd.cut(X['average_sales'],
                            bins=[-float('inf'), self.low_threshold, self.high_threshold, float('inf')],
                            labels=[1.0, 2.0, 3.0])
        X.drop("average_sales", axis=1,inplace=True)

        return pd.DataFrame(categories, index=X.index)
    
    
    def get_feature_names_out(self, input_features=None):
        return np.array(['county'])

X_train['average_sales'] = X_train['county'].map(county_sales_averages)

In [334]:


numerical_columns=X_train.select_dtypes(include=['int64', 'float64']).columns.to_list()
numerical_columns.remove('location_id')

categorical_columns= ["new_store", "transport_availability", "county"]

# Handling missing values

num_pipeline = make_pipeline(
SimpleImputer(strategy="median")
)

ordinal_pipeline= make_pipeline(
    OrdinalEncoder(categories=order)
                                )
onehot_pipeline= make_pipeline(
    OneHotEncoder(drop='if_binary')
                                )

county_encoder=CountySalesEncoder()
county_pipeline= make_pipeline(    
    CountySalesEncoder()
)



preprocessing = make_column_transformer(
(num_pipeline, numerical_columns),
(ordinal_pipeline,["transport_availability"]),
(onehot_pipeline,["new_store"]),
(county_pipeline,["county"]),
remainder='drop'
)

In [339]:

forest_reg = make_pipeline(preprocessing, RandomForestRegressor(n_estimators=1000, max_depth=10, random_state=42))
forest_rmses = -cross_val_score(forest_reg, X=X_train, y=y_train, scoring="neg_root_mean_squared_error", cv=10)


In [340]:
pd.Series(forest_rmses).describe()

count    10.000000
mean      0.495950
std       0.113788
min       0.349942
25%       0.419614
50%       0.472410
75%       0.613727
max       0.638082
dtype: float64

In [222]:
y_train.describe()

,normalised_sales
count,256.000000
mean,-0.003877
std,0.984346
min,-1.871568
25%,-0.560722
50%,-0.111056
75%,0.245952
max,2.968477


---

## Fine tuning the model 

In [344]:
# Fetching the input separately

X_train_prep=preprocessing.fit_transform(X_train,y_train)
preprocessing.get_feature_names_out()
X_train_prep=pd.DataFrame(X_train_prep,columns=preprocessing.get_feature_names_out())


In [345]:
# Model 1.1
# INCREASEING NUMBER OF TREES
forest_reg = make_pipeline(RandomForestRegressor(n_estimators=1500, max_depth=10, random_state=42))
forest_rmses = -cross_val_score(forest_reg, X=X_train_prep, y=y_train, scoring="neg_root_mean_squared_error", cv=10)
pd.Series(forest_rmses).describe()

count    10.000000
mean      0.378940
std       0.074160
min       0.308452
25%       0.325814
50%       0.353875
75%       0.406196
max       0.519871
dtype: float64

In [268]:
X_train_prep.columns

Index(['pipeline-1__crime_rate', 'pipeline-1__proportion_flats',
       'pipeline-1__proportion_nonretail', 'pipeline-1__commercial_property',
       'pipeline-1__household_size', 'pipeline-1__proportion_newbuilds',
       'pipeline-1__public_transport_dist', 'pipeline-1__property_value',
       'pipeline-1__school_proximity', 'pipeline-1__competitor_density',
       'pipeline-1__household_affluency', 'pipeline-2__transport_availability',
       'pipeline-3__new_store_yes', 'pipeline-4__county'],
      dtype='object')

In [346]:
# Model 1.2
# LOG TRANSFORMATION

columns_to_transform = ['pipeline-1__proportion_nonretail','pipeline-1__crime_rate','pipeline-1__property_value','pipeline-1__school_proximity','pipeline-1__competitor_density']
# log transformation
columns_to_consider=['pipeline-1__proportion_nonretail','pipeline-1__crime_rate','pipeline-1__property_value','pipeline-1__school_proximity','pipeline-1__competitor_density','pipeline-1__household_affluency','pipeline-1__household_size','pipeline-4__county','pipeline-1__public_transport_dist','pipeline-1__crime_rate','pipeline-1__proportion_newbuilds']
X_train_prep[columns_to_transform] = X_train_prep[columns_to_transform].apply(np.log)
forest_reg = make_pipeline(RandomForestRegressor(n_estimators= 1500, max_depth=10, random_state=42))
forest_rmses = -cross_val_score(forest_reg, X=X_train_prep, y=y_train, scoring="neg_root_mean_squared_error", cv=10)
pd.Series(forest_rmses).describe()

count    10.000000
mean      0.378620
std       0.072979
min       0.308170
25%       0.326965
50%       0.354506
75%       0.405828
max       0.514771
dtype: float64

In [350]:
# Model 1.2.2

# Outlier treatment
# 1. proportion_nonretail,
# 2. crime_rate,
# 3. property_value,
# 4. school_proximity


outlier_cols=['pipeline-1__proportion_nonretail', 'pipeline-1__crime_rate', 'pipeline-1__property_value', 'pipeline-1__school_proximity' ]
for col in outlier_cols:
    X_train_prep[col] = winsorize(X_train_prep[col], limits=[0.05, 0.05])

forest_reg = make_pipeline(RandomForestRegressor(n_estimators=1500, max_depth=10, random_state=42))
forest_rmses = -cross_val_score(forest_reg, X=X_train_prep, y=y_train, scoring="neg_root_mean_squared_error", cv=10)
pd.Series(forest_rmses).describe()

count    10.000000
mean      0.377662
std       0.072339
min       0.309652
25%       0.326239
50%       0.353346
75%       0.403179
max       0.515629
dtype: float64

In [ ]:
# Model 1.2.3

# log_transformation, Outlier treatment and radial basis transformation for treating bimodal distribution of crime_rate

outlier_cols=['pipeline-1__proportion_nonretail', 'pipeline-1__crime_rate', 'pipeline-1__property_value', 'pipeline-1__school_proximity' ]
for col in outlier_cols:
    X_train_prep[col] = winsorize(X_train_prep[col], limits=[0.05, 0.05])

forest_reg = make_pipeline(RandomForestRegressor(n_estimators=1500, max_depth=10, random_state=42))
forest_rmses = -cross_val_score(forest_reg, X=X_train_prep, y=y_train, scoring="neg_root_mean_squared_error", cv=10)
pd.Series(forest_rmses).describe()

count    10.000000
mean      0.377662
std       0.072339
min       0.309652
25%       0.326239
50%       0.353346
75%       0.403179
max       0.515629
dtype: float64

---

In [353]:

forest_reg.fit(X_train_prep, y_train)
rf_model = forest_reg.named_steps['randomforestregressor']

# Get the feature importances
feature_importances = rf_model.feature_importances_


fi=list(zip(X_train_prep.columns, feature_importances))
sorted_fi = sorted(fi, key=lambda x: x[1], reverse=True)
display(sorted_fi)

[('pipeline-4__county', 0.3358375824952366),
 ('pipeline-1__household_size', 0.2767432009075617),
 ('pipeline-1__household_affluency', 0.24124423550594365),
 ('pipeline-1__crime_rate', 0.03184489652586493),
 ('pipeline-1__public_transport_dist', 0.031647812396956784),
 ('pipeline-1__proportion_newbuilds', 0.026035683445684053),
 ('pipeline-1__property_value', 0.012981617329377885),
 ('pipeline-1__competitor_density', 0.012738800863235077),
 ('pipeline-1__school_proximity', 0.009927163062967464),
 ('pipeline-1__proportion_nonretail', 0.008427366822060687),
 ('pipeline-1__commercial_property', 0.007716838700266045),
 ('pipeline-2__transport_availability', 0.0024504157875947716),
 ('pipeline-1__proportion_flats', 0.0016304517287344584),
 ('pipeline-3__new_store_yes', 0.0007739344285157599)]

---

### Checking if feature engineering is showing positive impact - No

In [322]:
X_train_prep.columns

Index(['pipeline-1__crime_rate', 'pipeline-1__proportion_flats',
       'pipeline-1__proportion_nonretail', 'pipeline-1__commercial_property',
       'pipeline-1__household_size', 'pipeline-1__proportion_newbuilds',
       'pipeline-1__public_transport_dist', 'pipeline-1__property_value',
       'pipeline-1__school_proximity', 'pipeline-1__competitor_density',
       'pipeline-1__household_affluency', 'pipeline-2__transport_availability',
       'pipeline-3__new_store_yes', 'pipeline-4__county'],
      dtype='object')

In [323]:
X_train_prep['newbuild_property_value']=X_train_prep['pipeline-1__proportion_newbuilds']*X_train_prep['pipeline-1__property_value']
X_train_prep['flats_property_value']=X_train_prep['pipeline-1__proportion_flats']*X_train_prep['pipeline-1__property_value']
X_train_prep['commercial_property_value']=X_train_prep['pipeline-1__commercial_property']*X_train_prep['pipeline-1__property_value']
X_train_prep['senseful_school_proximity']=X_train_prep['pipeline-1__school_proximity']*X_train_prep['pipeline-1__proportion_flats']
X_train_prep['flats_crime_rate']=X_train_prep['pipeline-1__proportion_flats']*X_train_prep['pipeline-1__crime_rate']



In [328]:
X_train_prep

,pipeline-1__crime_rate,pipeline-1__proportion_flats,pipeline-1__proportion_nonretail,pipeline-1__commercial_property,pipeline-1__household_size,pipeline-1__proportion_newbuilds,pipeline-1__public_transport_dist,pipeline-1__property_value,pipeline-1__school_proximity,pipeline-1__competitor_density,pipeline-1__household_affluency,pipeline-2__transport_availability,pipeline-3__new_store_yes,pipeline-4__county,newbuild_property_value,flats_property_value,commercial_property_value,senseful_school_proximity,flats_crime_rate
0,0.367736,0.0,21.89,13.70,3.431,1.2,1.8125,437.0,19.0,396.90,3.8475,1.0,0.0,1.0,524.4,0.0,5986.9,0.0,0.000000
1,0.907062,0.0,8.14,9.40,2.456,63.4,3.7965,307.0,21.0,288.99,2.9225,1.0,0.0,1.0,19463.8,0.0,2885.8,0.0,0.000000
2,1.414523,0.0,8.14,9.40,2.570,1.9,3.7979,307.0,21.0,376.57,5.2550,1.0,0.0,1.0,583.3,0.0,2885.8,0.0,0.000000
3,10.552946,0.0,18.10,16.45,3.380,4.4,1.9682,666.0,20.2,60.72,6.0200,4.0,0.0,1.0,2930.4,0.0,10955.7,0.0,0.000000
4,0.431818,0.0,6.20,7.70,5.040,13.5,3.2157,307.0,17.4,387.38,0.7825,3.0,0.0,3.0,4144.5,0.0,2363.9,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0.183184,20.0,6.96,5.70,3.240,83.7,4.4290,223.0,19.0,396.90,1.6475,0.0,0.0,1.0,18665.1,4460.0,1271.1,380.0,3.663686
252,0.168732,25.0,5.13,5.15,2.741,33.8,7.2254,284.0,19.7,395.11,3.2875,3.0,0.0,1.0,9599.2,7100.0,1462.6,492.5,4.218290
253,0.144154,30.0,4.93,3.90,3.393,92.2,7.0355,300.0,16.6,374.71,1.2975,3.0,0.0,2.0,27660.0,9000.0,1170.0,498.0,4.324623
254,17.600541,0.0,18.10,9.40,1.519,0.0,1.6582,666.0,20.2,88.27,9.2450,4.0,0.0,1.0,0.0,0.0,6260.4,0.0,0.000000


In [324]:
outlier_cols=['pipeline-1__proportion_nonretail', 'pipeline-1__crime_rate', 'pipeline-1__property_value', 'pipeline-1__school_proximity' ]
for col in outlier_cols:
    X_train_prep[col] = winsorize(X_train_prep[col], limits=[0.05, 0.05])

forest_reg = make_pipeline(RandomForestRegressor(n_estimators=1000, max_depth=20, random_state=42))
forest_rmses = -cross_val_score(forest_reg, X=X_train_prep, y=y_train, scoring="neg_root_mean_squared_error", cv=10)
pd.Series(forest_rmses).describe()

count    10.000000
mean      0.379198
std       0.074005
min       0.312813
25%       0.327071
50%       0.350317
75%       0.399976
max       0.524046
dtype: float64